In [0]:
def get_silver_table_config(table_name):
    config_df = spark.sql(f"""
        SELECT source_system, silver_table_name, load_type, file_location
        FROM silver_control_table
        WHERE silver_table_name = '{table_name}'
    """)
    return config_df.collect()[0] if not config_df.isEmpty() else None

In [0]:
def log_to_table(table_name, layer, log_message, error_flag=False):
    timestamp = spark.sql("SELECT current_timestamp()").collect()[0][0]
    spark.sql(f"""
        INSERT INTO logging_table (table_name, layer, log_message, log_timestamp, error_flag)
        VALUES ('{table_name}', '{layer}', '{log_message}', '{timestamp}', {error_flag})
    """)

In [0]:
def extract_data(file_location):
    return spark.read.format("delta").load(file_location)

In [0]:
def load_to_silver_table(df, silver_table_name):
    df.write.format("delta").mode("overwrite").saveAsTable(silver_table_name)

In [0]:
# Get silver table names from the tracking table
def get_silver_table_names():
    silver_tables_df = spark.sql("""
        SELECT table_name 
        FROM tracking_table 
        WHERE layer = 'Silver'
    """)
    return [row.table_name for row in silver_tables_df.collect()]

In [0]:
def update_tracking_table(table_name, layer):
    timestamp = spark.sql("SELECT current_timestamp()").collect()[0][0]
    spark.sql(f"""
        MERGE INTO tracking_table AS t
        USING (SELECT '{table_name}' AS table_name, '{layer}' AS layer, '{timestamp}' AS last_loaded) AS s
        ON t.table_name = s.table_name AND t.layer = s.layer
        WHEN MATCHED THEN
            UPDATE SET last_loaded = s.last_loaded
        WHEN NOT MATCHED THEN
            INSERT (table_name, layer, last_loaded) VALUES (s.table_name, s.layer, s.last_loaded)
    """)

In [0]:
# Import the transformation function
dbutils.notebook.run("/Workspace/Metadata_Driven_ETL_Framework/Transformation_Logic/Transform_Silver_Layer", 600)

def execute_silver_tasks(table_names):
    for table_name in table_names:
        config = get_silver_table_config(table_name)
        if config:
            source_system = config.source_system
            silver_table_name = config.silver_table_name
            load_type = config.load_type
            file_location = config.file_location
            
            try:
                # Extract data
                df = extract_data(file_location)
                log_to_table(silver_table_name, "Silver", f"Successfully extracted data from {file_location}")

                # Transform data
                transformed_df = transform_data(df)
                log_to_table(silver_table_name, "Silver", "Successfully transformed data")

                # Load data into silver table
                load_to_silver_table(transformed_df, silver_table_name)
                log_to_table(silver_table_name, "Silver", "Successfully loaded data into silver table")

                # Update tracking table
                update_tracking_table(silver_table_name, "Silver")
                log_to_table(silver_table_name, "Silver", "Successfully updated tracking table")

            except Exception as e:
                log_to_table(silver_table_name, "Silver", f"Error processing {table_name}: {e}", error_flag=True)
                print(f"Error processing {table_name}: {e}")

        else:
            log_to_table(table_name, "Silver", f"No configuration found for table: {table_name}", error_flag=True)
            print(f"No configuration found for table: {table_name}")

In [0]:
try:
     # Get the list of silver table names
    silver_table_names = get_silver_table_names()
        
    # Execute the silver tasks
    execute_silver_tasks(silver_table_names)

except Exception as e:
    log_to_table("Silver Execution", "Silver", f"Error during silver table operations: {str(e)}", error_flag=True)
    print(f"Error during silver table operations: {str(e)}")